# Cálculo de deducciones incrementadas de Tramo 1 y Tramo 2

## Deducción incrementada Tramo 1

### A. Determinación de la remuneración bruta

In [2]:
#importo la info/librerias que voy a usar

import pandas as pd
from data import CWTR, CWTR_SUM, DOTACION, CUILES
from deducciones.incremental.calculo import *
from helpers.bases import tabla
import numpy as np
from remuneraciones.prorrateos import *
from parametrizacion.conceptos import ParamConceptos

In [12]:
#tomo la CWTR_SUM y me filtro aquellos conceptos que tengo que tener en cuenta

# t_prorrateos = tabla("prorrateos")
# t_prorrateos

# conceptos = tabla("conceptos")
# conceptos 

# conceptos_prorrateados = Prorrateos()
#conceptos_prorrateados.prorratear(CWTR_SUM)
# filtro = Prorrateos().filtrar(6066825 ,2, "cwtr")

conceptos = ParamConceptos()
conceptos

# Se crea una copia del df
df = CWTR_SUM.copy()

# Se agregan valores vacíos
df_vacio = []

# crea un df vacio para que ningun legajo quede sin valor en algun mes, por defecto le va poner cero
for empleado in DOTACION['legajo'].tolist():
    for mes in range(1,13):
        df_vacio.append({
            'legajo': empleado,
            'CCn': 'VACIO',
            'mes': mes,
            'importe': 0.00
        })

df_vacio = pd.DataFrame.from_records(df_vacio)
df = pd.concat([df_vacio, CWTR_SUM])


df['Suma'] = np.where(df['CCn'].isin(conceptos.listar('DedIncT1', '+')), df['importe'], 0)
df['Resta'] = np.where(df['CCn'].isin(conceptos.listar('DedIncT1', '-')), df['importe'], 0)
df['DeduccIncremT1'] = df['Suma'] - df['Resta']

df


,legajo,CCn,mes,importe,periodo,fecha_pago,Suma,Resta,DeduccIncremT1
0,6000002,VACIO,1,0.00,NaT,NaT,0.0,0.00,0.00
1,6000002,VACIO,2,0.00,NaT,NaT,0.0,0.00,0.00
2,6000002,VACIO,3,0.00,NaT,NaT,0.0,0.00,0.00
3,6000002,VACIO,4,0.00,NaT,NaT,0.0,0.00,0.00
4,6000002,VACIO,5,0.00,NaT,NaT,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...
483551,6114233,/361,12,1219.27,2022-12-31,2022-12-30,0.0,1219.27,-1219.27
483552,6114235,/351,12,1309.09,2022-12-31,2022-12-30,0.0,1309.09,-1309.09
483553,6114235,/361,12,1309.09,2022-12-31,2022-12-30,0.0,1309.09,-1309.09
483554,6114237,/351,12,1777.31,2022-12-31,2022-12-30,0.0,1777.31,-1777.31
